# Preprocessing and Data Extractions with GROBID
***

The purpose of this notebook is to process and extract information from a collection of MICCAI 2023 XML documents. The notebook organises this information into structured data formats (e.g., CSV files, dataframes) by:

1. Text wrapping utility function that wrappes text lines within a dataframe row.
2. Interacting with the GROBID client for processing documents (the client has to run via terminal before activated in the notebook).
3. Renaming MICCAI 2023 XML files based on their individual titles
4. Parsing XML files to extract headers and related text into a dataframe.
5. Aggregating and cleaning data from multiple sources.
6. Identifying specific content (e.g., papers related to cancer) within the aggregated data.

In [2]:
# Libraries 
import os
import re
import pandas as pd

# The GROBID client to process and extract information from XML files
from grobid_client.grobid_client import GrobidClient

# For parsing XML files
from xml.etree import ElementTree as et

1. Text wrapping utility function that wrappes text lines within a dataframe row
***

In [2]:
def wrap_text(text, width=80):
    if pd.isnull(text):
        return text
    
    wrapped_lines = []
    for paragraph in text.split('\n'):
        line = ''
        for word in paragraph.split():
            if len(line) + len(word) + 1 > width:
                wrapped_lines.append(line)
                line = word
            else:
                line += ' ' + word if line else word
        wrapped_lines.append(line)
    return '\n'.join(wrapped_lines)

2. Interacting with the GROBID client for processing documents 
***

In [3]:
def process_fulltext_document(client, process_file, output_dir):
    client.process('processFulltextDocument', process_file, output=output_dir, force=True)

3. Renaming MICCAI 2023 XML files based on their individual titles
***

In [ ]:

def rename_xml_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            try:
                tree = et.parse(file_path)
                root = tree.getroot()
                paper_title = find_title(root)
                if paper_title:
                    new_filename = paper_title.replace(" ", "_") + '.xml'
                    new_file_path = os.path.join(folder_path, new_filename)
                    os.rename(file_path, new_file_path)
            except et.ParseError as e:
                print(f"Error parsing '{filename}': {e}")

def find_title(element):
    if 'title' in element.tag.lower() and element.text:
        return element.text.strip()
    for child in element:
        title = find_title(child)
        if title:
            return title
    return None

4. Parsing XML files to extract headers and related text into a dataframe
***

In [ ]:

def parse_xml_and_extract_headers(file_path):
    tree = etree.parse(file_path)
    root = tree.getroot()
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Extract the paper title by XPath in the XML's structure
    paper_title_element = root.find('.//tei:title', ns)
    paper_title = paper_title_element.text if paper_title_element is not None else "No Title Found"

    headers = root.xpath('//tei:head', namespaces=ns)
    print(f"Found {len(headers)} headers in '{paper_title}'")
    
    data = []
    for header in headers:
        # Use XPath string() function to get all text within the <p> tags, including nested elements
        text_content = ''.join(header.getparent().xpath('.//tei:p//text()', namespaces=ns))
        data.append({
            'Paper Title': paper_title,
            'Header Number': header.get('n'),
            'Header Title': header.text,
            'Text': text_content  # Updated to use text_content
        })

    df = pd.DataFrame(data, columns=['Paper Title', 'Header Number', 'Header Title', 'Text'])
    return df

5. Aggregating and cleaning data from multiple sources.
***

In [3]:
def process_xml_folder(folder_path):
    # Aggregates data from multiple XML files in a given folder
    all_data_frames = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xml"):
            file_path = os.path.join(folder_path, file_name)
            df = parse_xml_and_extract_headers(file_path)
            all_data_frames.append(df)

    if all_data_frames:
        final_df = pd.concat(all_data_frames, ignore_index=True)
    else:
        final_df = pd.DataFrame()

    return final_df

# Clean dataframe from duplicates 
def clean_dataframe(df):
    df_cleaned = df.dropna(subset=['Header Title', 'Text'], how='all').dropna(subset=['Text'], how='any')
    return df_cleaned

# Merge each dataframe (where 1 dataframe contains all information from volume 1 etc.) into a final dataframe with all papers from MICCAI 2023
def merge_dataframes(cleaned_dataframes):
    for vol, df in cleaned_dataframes.items():
        volume_number = int(re.search(r'\d+', vol).group())
        df['Paper Title'] += f' (vol{volume_number})'
        df['Volume'] = volume_number
    return pd.concat(cleaned_dataframes.values(), ignore_index=True)

In [5]:
# Run GROBID in terminal before running the notebook
# Installation and running commands
# wget https://github.com/kermitt2/grobid/archive/0.8.0.zip
# unzip 0.8.0.zip
# cd grobid-0.8.0
# ./gradlew run

# GROBID library: Runs in terminal
grobid_server = 'http://localhost:8070'
client = GrobidClient(grobid_server=grobid_server)

# MICCAI 2023 PDF files, organised by volumes into separate folders
# 730 PDFs in total, divided into 10 folders
process_file = '/Users/yasminsarkhosh/Documents/GitHub/machine-learning-bsc-thesis-2024/miccai_2023'

# Output folder for processed PDF files as XML files
output_dir = './processed_documents'

# Call the function to start the preprocessing and data extraction 
process_fulltext_document(client, process_file, output_dir)

GROBID server does not appear up and running, the connection to the server failed


ServerUnavailableException: 